# Testing Notebook

### Imports

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
from scipy import interpolate
import time
from data_files import file_dict, runs_dict
from testing_class import testing
from range_estimator import range_est

### File Summary

In [ ]:

batt_cap = 58   # kWh; useable energy

file_summary = pd.DataFrame({
    'Run_ID': [],
    'Time (min)': [],
    'Max Speed (kts)': [],
    'Avg Speed (kts)': [],
    'Energy Expended (kWh)': [],
    'Distance Traveled (nm)': [],
    'Average Consumption (kWh/nm)': []
})

for run in runs_dict.keys():
    runs_dict[run]['Pack Voltage 1 V'] = runs_dict[run]['Pack Voltage 1 V']*10
    
    time_elapsed = runs_dict[run]['Time'].iloc[-1] - runs_dict[run]['Time'].iloc[0]
    speed_max = runs_dict[run]['Speed m/s'].max()*1.94384
    speed_avg = runs_dict[run]['Speed m/s'].mean()*1.94384
    battery_expended = runs_dict[run]['SOC 1 %'].iloc[0] - runs_dict[run]['SOC 1 %'].iloc[-1]
    energy_expended = battery_expended*batt_cap/100
    distance_traveled = (runs_dict[run]['Distance km'].iloc[-1] - runs_dict[run]['Distance km'].iloc[0])*0.539957
    avg_consumption = (energy_expended)/distance_traveled

    file_summary.loc[len(file_summary.index)] = [
        run, 
        round(time_elapsed.seconds/60, 1),
        round(speed_max, 1),
        round(speed_avg, 1),
        round(energy_expended, 2),
        round(distance_traveled, 2),
        round(avg_consumption, 1)
        ]

file_summary

### Individual Run Testing

In [ ]:
"""Import Data from file manager"""
df = testing().add_variables(runs_dict['Run 29'])   # Chose Run 29 because of interesting energy and distance values

'''Create algorithm instance'''
range_estimator = range_est(58, 2.5, 0.3, 0,0,0)
range_list = []

"""Test Loop"""
for i in range(len(df)):
    dataStream = testing().parse_csv(df.iloc[i])
    range_estimator.overall_avg(dataStream)
    range_list.append(range_estimator.range_remaining)

    print('Battery Remaining = %.1f percent | Range Remaining = %.1f nm' 
          % (dataStream['soc'], range_estimator.range_remaining), end=' \r')
    # time.sleep(.005)

duration = (df['Time'].iloc[-1] - df['Time'].iloc[0]).seconds/60
x = np.linspace(0, duration, len(range_list))
y = range_list

x_smooth = np.linspace(np.min(x), np.max(x), num = 20) #Num represents number of points, play around to affect smoothness
bspline = interpolate.make_interp_spline(x,y)
y_smooth = bspline(x_smooth)

fig, ax1 = plt.subplots(figsize=(12,5))

ax1.plot(x_smooth, y_smooth, color='red')
ax1.set_title('Range Remaining over Time')
ax1.set_xlabel('Time (mins)')
ax1.set_ylabel('Range Remaining (nm)')
ax1.legend(['Range Prediction'])
ax1.grid()

# ax2 = ax1.twinx()
# ax2.plot(x, df['tripDistance'], color='green')
# ax2.set_ylabel('Distance Traveled (nm)')
# ax2.legend(['Distance Traveled'], loc='upper center')
# ax2.invert_yaxis()

### Accuracy

In [ ]:
run_error, errors = testing().test_accuracy(df, range_list, interval=300)

x = range(len(errors))
y = errors

x_smooth = np.linspace(np.min(x),np.max(x),num = 50) #Num represents number of points, play around to affect smoothness
bspline = interpolate.make_interp_spline(x,y)
y_smooth = bspline(x_smooth)

plt.figure(figsize=(12,5))
plt.plot(x, y, x_smooth, y_smooth)
plt.title('Error of Range Prediction')
plt.xlabel('Comparison Points at each Interval')
plt.ylabel('Error (nm)')
plt.grid()

### Method Comparison, Batch Simulation

In [ ]:
# Values to be stored in inifile
batt_cap = 58
stored_dist_avg = 2.5
stored_time_avg = 0.3
n_runs = 0
roll_energy = 0
roll_distance = 0

# Class instances
test_inst = testing()
range_est = range_est(batt_cap,
                      stored_dist_avg,
                      stored_time_avg,
                      n_runs,
                      roll_energy,
                      roll_distance)

# lists to store accuracy of each run by different methods
accuracies1 = []
accuracies2 = []
accuracies3 = []

# Loop through runs
for run in runs_dict.keys():
    df = test_inst.add_variables(runs_dict[run])

    # lists to store range values
    range_dist_list = []
    range_time_list = []
    range_roll_list = []

    # Loop through rows of current run data
    for i in range(len(df)):
        # Simulate dataStream
        data = test_inst.parse_csv(df.iloc[i])

        # Record range remaining for each method
        range_est.overall_avg(data)
        range_dist_list.append(range_est.range_remaining)

        range_est.overall_time_avg(data)
        range_time_list.append(range_est.range_remaining)

        range_est.rolling_avg(data)
        range_roll_list.append(range_est.range_remaining)

    # Update averages once run is complete
    range_est.update_avg(data)

    # Check and record accuracy
    error_dist, errors_dist = test_inst.test_accuracy(df, range_dist_list)
    error_time, errors_time = test_inst.test_accuracy(df, range_time_list)
    error_roll, errors_roll = test_inst.test_accuracy(df, range_roll_list)

    accuracies1.append(error_dist)
    accuracies2.append(error_time)
    accuracies3.append(error_roll)

    print(run, 'Error', error_dist, error_time, error_roll, end=' \r')

# Create dataframe to store error values of each method for each run
error_summary = pd.DataFrame({
    'Run': runs_dict.keys(),
    'Overall Distance Average': accuracies1,
    'Overall Time Average': accuracies2,
    'Rolling Average': accuracies3
})


In [ ]:
error_summary

### Plot Method Errors

In [ ]:
print('\n', 
      'Distance Avg Error:    ', error_summary['Overall Distance Average'].mean(), '\n', 
      'Time Avg Error:        ', error_summary['Overall Time Average'].mean(), '\n',
      'Rolling Avg Error:     ', error_summary['Rolling Average'].mean())

x = [i + 1 for i in error_summary.index]
y1 = error_summary['Overall Distance Average']
y2 = error_summary['Overall Time Average']
y3 = error_summary['Rolling Average']

num = len(error_summary)*10
x_smooth = np.linspace(np.min(x), np.max(x), num)
bspline1 = interpolate.make_interp_spline(x, y1)
bspline2 = interpolate.make_interp_spline(x, y2)
bspline3 = interpolate.make_interp_spline(x, y3)
y1_smooth = bspline1(x_smooth)
y2_smooth = bspline2(x_smooth)
y3_smooth = bspline3(x_smooth)

plt.figure(figsize=(12,5))
plt.xlim(0, np.max(x))
plt.plot(x_smooth, y1_smooth,
         x_smooth, y2_smooth,
         x_smooth, y3_smooth
         )
plt.grid()
plt.legend(error_summary.columns[1:], loc='upper right')
plt.suptitle('Accuracy of different methods')
plt.xlabel('Run #')
plt.ylabel('Error (nm)')

## Extra

In [ ]:
# '''Summary Plots'''
# plt.figure(figsize=(12,5))

# plt.subplot(2,2,1)
# plt.plot(df.index, df['SOC 1 %'])
# plt.grid()
# plt.title('Charge (%) over Time')

# plt.subplot(2,2,2)
# plt.plot(df.index, df['Speed m/s']) #, df['Power 1 kW'])
# plt.grid()
# plt.title('Power (kW) and Speed (kts) over Time')

# plt.subplot(2,2,3)
# plt.plot(df.index, df['Pack Voltage 1 V'], df['Pack Current 1 A'])
# plt.grid()
# plt.title('Current and Voltage over Time')

# # plt.subplot(2,2,4)
# # plt.plot(df.index)
# # plt.grid()
# # plt.title('Empty')

# plt.subplots_adjust(hspace = 0.5)
# plt.subplots_adjust(wspace = 0.3)

# df.head()